In [1]:
import os
import math
import shutil
import numpy as np
from multiprocessing import Pool
from keras.optimizers import RMSprop, SGD
from keras.losses import categorical_crossentropy
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
GPU = '0'
RND = 1
RUN = 'B'
OUT_DIR = 'out'
TENSORBOARD_DIR = '/tensorboard/tf-speech-v2/%s_$fold$' % RUN
MODELS_DIR = '%s/models/run_%s/fold_$fold$' % (OUT_DIR, RUN)
INPUT_SIZE = (64, 64, 1)  # n_mels x width x 1ch
FOLDS = 10

In [3]:
%run '../data-generator.ipynb'
%run '../models.ipynb'

In [4]:
def choose_batch(n, train_X, train_Y, train_files, val_files):
    assert isinstance(val_files, set)

    # extra random indexes to search for files not in val_files
    def _extra_indexes():
        return np.random.randint(0, len(train_X), size=int(n * 0.15))

    ii = np.random.randint(0, len(train_X), size=n)
    extra_ii = []

    replaced = 0

    # replace indexes with files occuring in val_files
    for j in range(len(ii)):
        if '(silence)' != train_files[ii[j]]:
            while train_files[ii[j]] in val_files:
                if len(extra_ii) == 0: extra_ii = _extra_indexes()
                ii[j], extra_ii = extra_ii[0], extra_ii[1:]
                replaced += 1

    X = train_X[ii]
    Y = train_Y[ii]
    files = train_files[ii]
    
    return X, Y, files

In [5]:
train_X = np.memmap('%s/train_X.mem' % OUT_DIR, np.float32,
                    'r').reshape((-1, ) + INPUT_SIZE)
train_Y = np.memmap('%s/train_Y.mem' % OUT_DIR, np.float32, 'r').reshape(
    (-1, len(LABELS)))

train_files = np.load('%s/train_files.npy' % OUT_DIR)

assert len(train_Y) == len(train_X)
assert len(train_files) == len(train_X)

print('len(train_X):', len(train_X))

len(train_X): 1000000


In [6]:
# training params
N_PER_BATCH = 1000
# last number splits train set into XX epochs
STEPS_PER_EPOCH = len(train_X) // N_PER_BATCH // 10
N_EPOCHS = 100

In [7]:
test_X = np.memmap('%s/test/test_X.mem' % (OUT_DIR), np.float32,
                   'r').reshape((-1, ) + INPUT_SIZE)

In [8]:
for fold in range(FOLDS):

    print('fold:', fold)

    # read val data
    val_X = np.load('%s/val/val_X_%d.npy' % (OUT_DIR, fold))
    val_Y = np.load('%s/val/val_Y_%d.npy' % (OUT_DIR, fold))
    val_files = np.load('%s/val/val_files_%d.npy' % (OUT_DIR, fold))
    assert len(val_X) == len(val_files)
    assert len(val_Y) == len(val_files)
    print('len(val_X):', len(val_X))
    val_files = set(val_files)

    # create dir to store models
    models_dir = MODELS_DIR.replace('$fold$', str(fold))
    os.makedirs(models_dir, exist_ok=True)
    print('models_dir:', models_dir)

    def train_generator(n_per_batch):
        while True:
            X, Y, files = choose_batch(n_per_batch, train_X, train_Y,
                                       train_files, val_files)
            yield (X, Y)

    # rm/create tensorboard dir
    tensorboard_dir = TENSORBOARD_DIR.replace('$fold$', str(fold))
    shutil.rmtree(tensorboard_dir, ignore_errors=True)
    os.makedirs(tensorboard_dir)
    print('tensorboard_dir:', tensorboard_dir)

    # create model
    model = Model_3(input_size=INPUT_SIZE, output_size=len(LABELS))
    model.build()
    optimizer = RMSprop(lr=1e-3)
    model.m.compile(
        optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy']\
    )

    model.m.fit_generator(
        train_generator(N_PER_BATCH),
        STEPS_PER_EPOCH,
        epochs=N_EPOCHS,
        validation_data=(val_X, val_Y),
        callbacks=[
            TensorBoard(log_dir=tensorboard_dir),
            ModelCheckpoint(
                models_dir +
                '/e{epoch:03d}-l={loss:.5f}-vl={val_loss:.5f}-a={acc:.5f}-va={val_acc:.5f}.h5',
                monitor='val_acc',
                verbose=0,
                save_best_only=True,
                save_weights_only=False,
                mode='auto'),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.2,
                patience=1,
                min_lr=1e-9,
                verbose=1),
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.000001,
                patience=5,
                verbose=1,
                mode='auto')
        ])

    # predict on holdout
    holdout_X = np.load('%s/holdout/holdout_X.npy' % (OUT_DIR))
    holdout_Y = np.load('%s/holdout/holdout_Y.npy' % (OUT_DIR))
    hp = model.m.predict(holdout_X)
    np.save('%s/holdout/holdout_predictions_%d.npy' % (OUT_DIR, fold), hp)

    # eval on holdout
    print('evaluation on holdout:')
    print(model.m.metrics_names)
    print(model.m.evaluate(holdout_X, holdout_Y, verbose=0))

    # predict on test data
    test_predictions = model.m.predict(test_X, verbose=1, batch_size=1000)
    np.save('%s/test/test_predictions_%d.npy' % (OUT_DIR, fold),
            test_predictions)

    print('')

fold: 0
len(val_X): 2441
models_dir: out/models/run_B/fold_0
tensorboard_dir: /tensorboard/tf-speech-v2/B_0
Epoch 1/100
100/100 [==============================] - 49s - loss: 2.5755 - acc: 0.1014 - val_loss: 2.4481 - val_acc: 0.0995
Epoch 2/100
100/100 [==============================] - 47s - loss: 2.1545 - acc: 0.2677 - val_loss: 1.4055 - val_acc: 0.5166
Epoch 3/100
100/100 [==============================] - 47s - loss: 1.3632 - acc: 0.5514 - val_loss: 0.5919 - val_acc: 0.7948
Epoch 4/100
100/100 [==============================] - 47s - loss: 0.8723 - acc: 0.7187 - val_loss: 0.3570 - val_acc: 0.8964
Epoch 5/100
100/100 [==============================] - 47s - loss: 0.6290 - acc: 0.8024 - val_loss: 0.3106 - val_acc: 0.9099
Epoch 6/100
100/100 [==============================] - 47s - loss: 0.4882 - acc: 0.8466 - val_loss: 0.2332 - val_acc: 0.9263
Epoch 7/100
100/100 [==============================] - 47s - loss: 0.3860 - acc: 0.8792 - val_loss: 0.2221 - val_acc: 0.9398
Epoch 8/100
100/1

100/100 [==============================] - 47s - loss: 1.1741 - acc: 0.6119 - val_loss: 0.5498 - val_acc: 0.8139
Epoch 4/100
100/100 [==============================] - 47s - loss: 0.7943 - acc: 0.7437 - val_loss: 0.3367 - val_acc: 0.8943
Epoch 5/100
100/100 [==============================] - 47s - loss: 0.5900 - acc: 0.8126 - val_loss: 0.3153 - val_acc: 0.9164
Epoch 6/100
100/100 [==============================] - 47s - loss: 0.4522 - acc: 0.8578 - val_loss: 0.2723 - val_acc: 0.9283
Epoch 7/100
100/100 [==============================] - 47s - loss: 0.3558 - acc: 0.8900 - val_loss: 0.2713 - val_acc: 0.9316
Epoch 8/100
100/100 [==============================] - 47s - loss: 0.2873 - acc: 0.9118 - val_loss: 0.2900 - val_acc: 0.9328
Epoch 9/100
 99/100 [============================>.] - ETA: 0s - loss: 0.2198 - acc: 0.9328
Epoch 00008: reducing learning rate to 0.00020000000949949026.
100/100 [==============================] - 47s - loss: 0.2198 - acc: 0.9327 - val_loss: 0.2864 - val_acc: 0

100/100 [==============================] - 47s - loss: 1.0741 - acc: 0.6499 - val_loss: 0.6608 - val_acc: 0.7680
Epoch 4/100
100/100 [==============================] - 47s - loss: 0.7737 - acc: 0.7521 - val_loss: 0.3097 - val_acc: 0.9008
Epoch 5/100
100/100 [==============================] - 47s - loss: 0.5721 - acc: 0.8182 - val_loss: 0.2492 - val_acc: 0.9225
Epoch 6/100
100/100 [==============================] - 47s - loss: 0.4542 - acc: 0.8579 - val_loss: 0.2899 - val_acc: 0.9209
Epoch 7/100
100/100 [==============================] - 47s - loss: 0.3481 - acc: 0.8903 - val_loss: 0.2441 - val_acc: 0.9389
Epoch 8/100
100/100 [==============================] - 47s - loss: 0.2771 - acc: 0.9143 - val_loss: 0.2570 - val_acc: 0.9414
Epoch 9/100
100/100 [==============================] - 47s - loss: 0.2242 - acc: 0.9307 - val_loss: 0.2176 - val_acc: 0.9516
Epoch 10/100
100/100 [==============================] - 47s - loss: 0.1749 - acc: 0.9454 - val_loss: 0.2751 - val_acc: 0.9398
Epoch 11/10

100/100 [==============================] - 48s - loss: 1.7570 - acc: 0.4155 - val_loss: 0.7429 - val_acc: 0.7594
Epoch 3/100
100/100 [==============================] - 48s - loss: 1.1038 - acc: 0.6394 - val_loss: 0.5029 - val_acc: 0.8311
Epoch 4/100
100/100 [==============================] - 48s - loss: 0.7654 - acc: 0.7561 - val_loss: 0.4316 - val_acc: 0.8758
Epoch 5/100
100/100 [==============================] - 48s - loss: 0.5692 - acc: 0.8208 - val_loss: 0.3408 - val_acc: 0.9012
Epoch 6/100
100/100 [==============================] - 47s - loss: 0.4317 - acc: 0.8650 - val_loss: 0.3541 - val_acc: 0.8975
Epoch 7/100
100/100 [==============================] - 47s - loss: 0.3442 - acc: 0.8929 - val_loss: 0.3248 - val_acc: 0.9209
Epoch 8/100
100/100 [==============================] - 47s - loss: 0.2694 - acc: 0.9163 - val_loss: 0.2904 - val_acc: 0.9377
Epoch 9/100
100/100 [==============================] - 47s - loss: 0.2079 - acc: 0.9360 - val_loss: 0.2807 - val_acc: 0.9324
Epoch 10/100